In [9]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 15.1MB/s 
     |████████████████████████████████| 3.0MB 23.2MB/s 
     |████████████████████████████████| 1.1MB 32.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=54c33a6b090189e73631ae06f8a4927ba798f481a1130302592e62a57667782f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# import pandas as pd

# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv', sep='|')
# df = df[df.Intencion != 'Cat_104']

# df.columns = ['texts','labels']

# from sklearn.model_selection import train_test_split

# #/content/drive/My Drive/Colab Notebooks/test.csv
# train, val = train_test_split(df, test_size=0.15, stratify= df.labels)

# train.to_csv('/content/drive/My Drive/Colab Notebooks/train_split_santander.csv', index=False)
# val.to_csv('/content/drive/My Drive/Colab Notebooks/val_split_santander.csv', index=False)

In [5]:
# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_santander.csv')
# df.columns = ['id','texts']
# df.to_csv('/content/drive/My Drive/Colab Notebooks/test_santander_texts.csv', index=False)

In [12]:
import pandas as pd
import uuid
import os
import random
from argparse import Namespace

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.utils.data.distributed import DistributedSampler

from tqdm import tqdm, trange

from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer)
from transformers import AdamW #, WarmupLinearSchedule from optimization
from transformers import WarmUp as WarmupLinearSchedule

MODEL_CLASSES = { 'bert': (BertConfig, BertForSequenceClassification, BertTokenizer) }
import logging

import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [13]:
args = Namespace(
    n_gpu=1,
    seed=1337,
    train_batch_size=16,
    per_gpu_train_batch_size=16,
    per_gpu_eval_batch_size=16,
    local_rank=-1,
    max_seq_length=128,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    num_train_epochs=4.0,
    max_steps=-1,
    warmup_steps=0,
    model_type='bert',
    data_dir='/content/drive/My Drive/Colab Notebooks/',
    output_dir='./output/',
    train_filepath='/content/drive/My Drive/Colab Notebooks/',
    valid_filepath='/content/drive/My Drive/Colab Notebooks/',
    test_filepath='/content/drive/My Drive/Colab Notebooks/',
    config_name='dccuchile/bert-base-spanish-wwm-uncased',
    tokenizer_name='dccuchile/bert-base-spanish-wwm-uncased', #dccuchile/bert-base-spanish-wwm-uncased
    do_lower_case=True,
    cuda=True,
)

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

args.train_filepath = os.path.join(args.data_dir, 'train_split_santander.csv')
args.valid_filepath=os.path.join(args.data_dir, 'val_split_santander.csv')
args.test_filepath=os.path.join(args.data_dir, 'test_santander_texts.csv')


Using CUDA: True


## data preparation: utility class

In [4]:

class InputExample(object):
    """
    A single training/test example for simple sequence classification.

    Args:
        guid: Unique id for the example.
        text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
        text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    def __init__(self, guid, text_a, text_b=None, label=None):
        self.guid   = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label  = label

class InputFeatures(object):
    """
    A single set of features of data.

    Args:
        input_ids: Indices of input sequence tokens in the vocabulary.
        attention_mask: Mask to avoid performing attention on padding token indices.
            Mask values selected in ``[0, 1]``:
            Usually  ``1`` for tokens that are NOT MASKED, ``0`` for MASKED (padded) tokens.
        token_type_ids: Segment token indices to indicate first and second portions of the inputs.
        label: Label corresponding to the input
    """

    def __init__(self, input_ids, attention_mask, token_type_ids, label):
        self.input_ids      = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label          = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
        
    def get_test_examples(self):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError() 

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()


In [20]:
class MultiClassProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def __init__(self, train_filepath, dev_filepath, test_filepath):
        self.train_filepath = train_filepath
        self.dev_filepath   = dev_filepath
        self.test_filepath  = test_filepath

    def get_train_examples(self):
        """See base class."""
        df            = self._get_dataframe(self.train_filepath)
        return self._get_examples(df)

    def get_dev_examples(self):
        """See base class."""
        df            = self._get_dataframe(self.dev_filepath)
        return self._get_examples(df)
    
    def get_test_examples(self):
        """Gets a collection of `InputExample`s for the dev set."""
        df            = self._get_dataframe(self.test_filepath)
        # return self._get_examples(df)
        examples = []
        for index, row in df.iterrows():
            examples.append(InputExample(guid=str(uuid.uuid4()), text_a=row['texts'], text_b=None, label=None))
        return examples

    def get_labels(self):
        """See base class."""
        df            = pd.read_csv(self.train_filepath)
        self.labels   = list(df.labels.unique())
        return self.labels
    
    def _get_dataframe(self, filepath):
        df            = pd.read_csv(filepath)
        return df

    def _get_examples(self, df):
        examples = []
        for index, row in df.iterrows():
            examples.append(InputExample(guid=str(uuid.uuid4()), text_a=row['texts'], text_b=None, label=row['labels']))
        return examples

## helper functions

In [6]:

def convert_examples_to_features(examples, tokenizer,
                                      max_length=128,
                                      task=None,
                                      label_list=None,
                                      output_mode=None, 
                                      pad_on_left=False,
                                      pad_token=0,
                                      pad_token_segment_id=0,
                                      mask_padding_with_zero=True):
    """
    Loads a data file into a list of ``InputFeatures``

    Args:
        examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
        tokenizer: Instance of a tokenizer that will tokenize the examples
        max_length: Maximum example length
        task: GLUE task
        label_list: List of labels. Can be obtained from the processor using the ``processor.get_labels()`` method
        output_mode: String indicating the output mode. Either ``regression`` or ``classification``
        pad_on_left: If set to ``True``, the examples will be padded on the left rather than on the right (default)
        pad_token: Padding token
        pad_token_segment_id: The segment ID for the padding token (It is usually 0, but can vary such as for XLNet where it is 4)
        mask_padding_with_zero: If set to ``True``, the attention mask will be filled by ``1`` for actual values
            and by ``0`` for padded values. If set to ``False``, inverts it (``1`` for padded values, ``0`` for
            actual values)

    Returns:
        If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
        containing the task-specific features. If the input is a list of ``InputExamples``, will return
        a list of task-specific ``InputFeatures`` which can be fed to the model.

    """
    
    if task is not None:
        processor = glue_processors[task]()
        if label_list is None:
            label_list = processor.get_labels()
            logger.info("Using label list %s for task %s" % (label_list, task))
        if output_mode is None:
            output_mode = glue_output_modes[task]
            logger.info("Using output mode %s for task %s" % (output_mode, task))

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d" % (ex_index))

        inputs = tokenizer.encode_plus(
            example.text_a,
            example.text_b,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
            token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)

        assert len(input_ids) == max_length, "Error with input length {} vs {}".format(len(input_ids), max_length)
        assert len(attention_mask) == max_length, "Error with input length {} vs {}".format(len(attention_mask), max_length)
        assert len(token_type_ids) == max_length, "Error with input length {} vs {}".format(len(token_type_ids), max_length)

        label = label_map[example.label]

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("attention_mask: %s" % " ".join([str(x) for x in attention_mask]))
            logger.info("token_type_ids: %s" % " ".join([str(x) for x in token_type_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label))

        features.append(
                InputFeatures(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids,
                              label=label))
    return features

In [7]:
def get_examples_dataset(examples, labels, tokenzier):
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=labels,
                                            max_length=args.max_seq_length,
                                            pad_on_left=False,
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=0,
    )


    # Convert to Tensors and build dataset
    all_input_ids       = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask  = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids  = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels          = torch.tensor([f.label for f in features], dtype=torch.long)
    
    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

    return dataset

## start of program

In [21]:
processor       = MultiClassProcessor(args.train_filepath, args.valid_filepath, args.test_filepath)
label_list      = processor.get_labels()
train_examples  = processor.get_train_examples()
eval_examples   = processor.get_dev_examples()
test_examples   = processor.get_test_examples()

In [15]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config       = config_class.from_pretrained(args.config_name, num_labels=len(label_list))
tokenizer    = tokenizer_class.from_pretrained(args.tokenizer_name, do_lower_case=args.do_lower_case)
model        = model_class.from_pretrained(args.config_name, config=config).to(args.device)


07/07/2020 03:11:02 - INFO - filelock -   Lock 139663501763416 acquired on /root/.cache/torch/transformers/9da2d8d1512c7ec3ed152214a324d14f2f68e2ca15cdf4bedf01d9062ff295bc.e75c4c740793050b7687a1d6c2b9a136a2fbfe439e9fe9a1ba6ddb7d5db29c23.lock
07/07/2020 03:11:02 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp1944jj07


07/07/2020 03:11:02 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/config.json in cache at /root/.cache/torch/transformers/9da2d8d1512c7ec3ed152214a324d14f2f68e2ca15cdf4bedf01d9062ff295bc.e75c4c740793050b7687a1d6c2b9a136a2fbfe439e9fe9a1ba6ddb7d5db29c23
07/07/2020 03:11:02 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/9da2d8d1512c7ec3ed152214a324d14f2f68e2ca15cdf4bedf01d9062ff295bc.e75c4c740793050b7687a1d6c2b9a136a2fbfe439e9fe9a1ba6ddb7d5db29c23
07/07/2020 03:11:02 - INFO - filelock -   Lock 139663501763416 released on /root/.cache/torch/transformers/9da2d8d1512c7ec3ed152214a324d14f2f68e2ca15cdf4bedf01d9062ff295bc.e75c4c740793050b7687a1d6c2b9a136a2fbfe439e9fe9a1ba6ddb7d5db29c23.lock
07/07/2020 03:11:02 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-span

07/07/2020 03:11:03 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/vocab.txt in cache at /root/.cache/torch/transformers/7955a302c4e3296b0c9697ebe339ba914d74bd65797fabbc5b119f763358172e.8f03a158e77bbe8af415df188da5da729e841544f66dc9fcce5db29a9b530f09
07/07/2020 03:11:03 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/7955a302c4e3296b0c9697ebe339ba914d74bd65797fabbc5b119f763358172e.8f03a158e77bbe8af415df188da5da729e841544f66dc9fcce5db29a9b530f09
07/07/2020 03:11:03 - INFO - filelock -   Lock 139663501845336 released on /root/.cache/torch/transformers/7955a302c4e3296b0c9697ebe339ba914d74bd65797fabbc5b119f763358172e.8f03a158e77bbe8af415df188da5da729e841544f66dc9fcce5db29a9b530f09.lock
07/07/2020 03:11:03 - INFO - filelock -   Lock 139666285613520 acquired on /root/.cache/torch/transformers/9223ec2262c881535d6847ce9da4b71264b71ad556c77c784e4b51ff1f5f6565

07/07/2020 03:11:03 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/added_tokens.json in cache at /root/.cache/torch/transformers/9223ec2262c881535d6847ce9da4b71264b71ad556c77c784e4b51ff1f5f6565.3889713104075cfee9e96090bcdd0dc753733b3db9da20d1dd8b2cd1030536a2
07/07/2020 03:11:03 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/9223ec2262c881535d6847ce9da4b71264b71ad556c77c784e4b51ff1f5f6565.3889713104075cfee9e96090bcdd0dc753733b3db9da20d1dd8b2cd1030536a2
07/07/2020 03:11:03 - INFO - filelock -   Lock 139666285613520 released on /root/.cache/torch/transformers/9223ec2262c881535d6847ce9da4b71264b71ad556c77c784e4b51ff1f5f6565.3889713104075cfee9e96090bcdd0dc753733b3db9da20d1dd8b2cd1030536a2.lock
07/07/2020 03:11:03 - INFO - filelock -   Lock 139663501762856 acquired on /root/.cache/torch/transformers/628ac9b20df95e32076840e01abe63614cbaa8545ba0bdd41ece5732

07/07/2020 03:11:03 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/special_tokens_map.json in cache at /root/.cache/torch/transformers/628ac9b20df95e32076840e01abe63614cbaa8545ba0bdd41ece5732c510ce74.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
07/07/2020 03:11:03 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/628ac9b20df95e32076840e01abe63614cbaa8545ba0bdd41ece5732c510ce74.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
07/07/2020 03:11:04 - INFO - filelock -   Lock 139663501762856 released on /root/.cache/torch/transformers/628ac9b20df95e32076840e01abe63614cbaa8545ba0bdd41ece5732c510ce74.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
07/07/2020 03:11:04 - INFO - filelock -   Lock 139666285613520 acquired on /root/.cache/torch/transformers/fefcf38d9ec67bb1fd312a883b6e3ea8173911159c2c349f4b

07/07/2020 03:11:04 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-spanish-wwm-uncased/tokenizer_config.json in cache at /root/.cache/torch/transformers/fefcf38d9ec67bb1fd312a883b6e3ea8173911159c2c349f4b7e32f8ed32f99f.83cc14b7f51b5cfec42504ee7d78ef124a5cc546509212bd000e707b50a8ccb3
07/07/2020 03:11:04 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/fefcf38d9ec67bb1fd312a883b6e3ea8173911159c2c349f4b7e32f8ed32f99f.83cc14b7f51b5cfec42504ee7d78ef124a5cc546509212bd000e707b50a8ccb3
07/07/2020 03:11:04 - INFO - filelock -   Lock 139666285613520 released on /root/.cache/torch/transformers/fefcf38d9ec67bb1fd312a883b6e3ea8173911159c2c349f4b7e32f8ed32f99f.83cc14b7f51b5cfec42504ee7d78ef124a5cc546509212bd000e707b50a8ccb3.lock
07/07/2020 03:11:04 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/dccuchile/bert-base-span

07/07/2020 03:11:04 - INFO - filelock -   Lock 139663501765600 acquired on /root/.cache/torch/transformers/45250002ed80e0cb4fbeac9290d218e37e686ba9c42fe58bb4ec5af5e4d60aa9.9f64aa40cdc26c80eeb1ba80c929ec27ae13318c5647290c04eaf2a289d9ad30.lock
07/07/2020 03:11:04 - INFO - transformers.file_utils -   https://cdn.huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp5acjwe4b


07/07/2020 03:11:14 - INFO - transformers.file_utils -   storing https://cdn.huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/pytorch_model.bin in cache at /root/.cache/torch/transformers/45250002ed80e0cb4fbeac9290d218e37e686ba9c42fe58bb4ec5af5e4d60aa9.9f64aa40cdc26c80eeb1ba80c929ec27ae13318c5647290c04eaf2a289d9ad30
07/07/2020 03:11:14 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/45250002ed80e0cb4fbeac9290d218e37e686ba9c42fe58bb4ec5af5e4d60aa9.9f64aa40cdc26c80eeb1ba80c929ec27ae13318c5647290c04eaf2a289d9ad30
07/07/2020 03:11:14 - INFO - filelock -   Lock 139663501765600 released on /root/.cache/torch/transformers/45250002ed80e0cb4fbeac9290d218e37e686ba9c42fe58bb4ec5af5e4d60aa9.9f64aa40cdc26c80eeb1ba80c929ec27ae13318c5647290c04eaf2a289d9ad30.lock
07/07/2020 03:11:14 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/pytorch_model.bin from cache at /root/.

07/07/2020 03:11:19 - WARNING - transformers.modeling_utils -   Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
07/07/2020 03:11:19 - WARNING - transformers.modeling_utils -   Some 

## training and evaluation helper functions

In [16]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return {"acc": simple_accuracy(preds, labels)}

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

In [15]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    args.train_batch_size   = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler           = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader        = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    t_total                 = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    #scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    #scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps)

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0])
    set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[3]}
            if args.model_type != 'distilbert':
                inputs['token_type_ids'] = batch[2] if args.model_type in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                #scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1
                
                if global_step % 100 == 0:
                    output_dir = os.path.join(args.output_dir, 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(args.output_dir)
                    
                    torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                    logger.info("Saving model checkpoint to %s", output_dir)
                    
    # save 
    return global_step, tr_loss / global_step



def evaluate(args, eval_dataset, model, tokenizer):
    results = {}
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****")
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[3]}
            if args.model_type != 'distilbert':
                inputs['token_type_ids'] = batch[2] if args.model_type in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            
    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)

    result = compute_metrics("eval_task", preds, out_label_ids)
    results.update(result)
        
    return results

def make_predictions(args, test_dataset, model, tokenizer):
    results = {}
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(test_dataset) if args.local_rank == -1 else DistributedSampler(test_dataset)
    eval_dataloader = DataLoader(test_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running prediction {} *****")
    logger.info("  Num examples = %d", len(test_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    #eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Predicting"):
        #model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1]}
            if args.model_type != 'distilbert':
                inputs['token_type_ids'] = batch[2] if args.model_type in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
            outputs = model(**inputs)
            logits = outputs[:2]

            #eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        if preds is None:
            preds = logits.detach().cpu().numpy()
            #out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            #out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            
    #eval_loss = eval_loss / nb_eval_steps
    #######preds = np.argmax(preds, axis=1)############VER

    #result = compute_metrics("eval_task", preds, out_label_ids)
    #results.update(result)
        
    return preds

## start of training

In [16]:
set_seed(args)
train_dataset = get_examples_dataset(train_examples, label_list, tokenizer)
global_step, tr_loss = train(args, train_dataset, model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


07/07/2020 00:45:48 - INFO - __main__ -   Writing example 0
07/07/2020 00:45:48 - INFO - __main__ -   *** Example ***
07/07/2020 00:45:48 - INFO - __main__ -   guid: aa0bba3b-4e65-4f35-bcd5-0848ccaf5d0f
07/07/2020 00:45:48 - INFO - __main__ -   input_ids: 4 2418 1032 1641 1105 6342 1009 4009 18808 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
07/07/2020 00:45:48 - INFO - __main__ -   attention_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/07/2020 00:45:48 - INFO - __main__ -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## load generated model for evalution

In [17]:
checkpoint   = os.path.join(args.output_dir, 'checkpoint-4200')
tokenizer1    = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
eval_dataset = get_examples_dataset(eval_examples, label_list, tokenizer)
model1        = model_class.from_pretrained(checkpoint).to(args.device)



07/07/2020 03:11:35 - INFO - transformers.tokenization_utils_base -   Model name './output/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming './output/' is a path, a model identifier, or url to a directory containing tokenizer files.


OSError: ignored

## start of evaluation

In [18]:
result       = evaluate(args, eval_dataset, model1, tokenizer1)
logger.info(" evaluation result = %s", result)



NameError: ignored

# Submit



In [2]:
checkpoint   = os.path.join(args.output_dir, 'checkpoint-4200')
tokenizer1    = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
test_dataset = get_examples_dataset(test_examples, label_list, tokenizer)
model1        = model_class.from_pretrained(checkpoint).to(args.device)

NameError: ignored

In [ ]:
preds = make_predictions(args, test_dataset, model1, tokenizer1)

In [ ]:
#!pip install 'transformers==2.0.0' --force-reinstall